In [ ]:
import stir

In [ ]:
from odlpet.stir.io import volume_from_file

In [ ]:
from pathlib import Path
data_path = Path() / 'data' / 'stir'

In [ ]:
volume_path = data_path / 'FDG_g1.hv'

In [ ]:
volume = volume_from_file(volume_path.as_posix())

In [ ]:
%matplotlib inline

In [ ]:
volume.show(coords=(10,None,None), aspect='equal')

## Read the data

In [ ]:
prompt_path = data_path / 'my_prompts_g1.hs'

In [ ]:
data = stir.ProjData.read_from_file(prompt_path.as_posix())

In [ ]:
pinfo = data.get_proj_data_info()

In [ ]:
from odlpet.scanner.compression import Compression

### Without guessing

One can get the right dimension directly using:

In [ ]:
proj_ = comp.get_projector(stir_proj_data_info=pinfo)

In [ ]:
proj_.range.shape

### Guessing the compression

Here we have to guess the compression level from the shape of the data, which is:

In [ ]:
data.to_array().shape()

First, create a `Compression` object with the right scanner:

In [ ]:
comp = Compression.from_stir_proj_data_info(pinfo)

There are 15 sinograms, for 8 rings, which suggests the compression 3 and that there is just one segment (i.e., only direct sinograms).

In [ ]:
comp.span_num = 3

In [ ]:
comp.max_num_segments = 1

Next, the resolution is 64 view angles, and 192 tangential positions:

In [ ]:
comp.num_of_views = 64
comp.num_non_arccor_bins = 192

In [ ]:
proj = comp.get_projector()

In [ ]:
proj.range.shape

We see that the range has the right shape.

In [ ]:
import stirextra

In [ ]:
d = proj.range.element(stirextra.to_numpy(data))

In [ ]:
d.show(indices=[comp.get_offset(0,5),None,None])

## Solving

In [ ]:
from odl.solvers.iterative.statistical import mlem

In [ ]:
recon = proj.domain.one()*.1

In [ ]:
mlem(proj, recon, d, niter=10)

In [ ]:
recon.show(coords=(5,None,None), aspect='equal')